In [23]:
import pdfplumber
import math

def open(file):
    def getVndData(**pos_vnd):
        finalPrice = get_text(p0,pos_vnd['finalPrice'])
        if finalPrice != None:
            finalPrice= finalPrice.replace('$', '')
        
        d = {
            'sn':get_text(p0,pos_vnd['sn']),
            'id':get_text(p0,pos_vnd['id']),
            'name':get_text(p0,pos_vnd['name']),
            'type':get_text(p0,pos_vnd['type']),
            'quo':get_text(p0,pos_vnd['quo']).replace('$', ''),
            'quoRate':get_text(p0,pos_vnd['quoRate']),
            'finalPrice':finalPrice,
            'finalRate':get_text(p0,pos_vnd['finalRate'])
        }
        
        return d
    
    pdf_1 = pdfplumber.open(file)
    p0 = pdf_1.pages[0]

    pos_caseID = {'x0':22.320, 'top':91.744, 'x1':70.320, 'bottom':109.600}
    pos_caseName = {'x0':29.280, 'top':135.104, 'x1':393.600, 'bottom':152.960}
    pos_sn = {'x0':81.320, 'top':91.744, 'x1':93.320, 'bottom':109.600}
    pos_comp = {'x0':100.360, 'top':91.744, 'x1': 118.360, 'bottom':109.600}
    pos_budget = {'x0':700.750, 'top':91.304, 'x1': 815.87, 'bottom':109.16}
    pos_type = {'x0':175.960, 'top':91.744, 'x1': 199.960, 'bottom':109.600}
    pos_currency ={'x0': 715.750, 'top': 49.304, 'x1':751.750, 'bottom':67.160}
    
    pos_vendorID= {'x0':46.790, 'top':193.628, 'x1':127.790, 'bottom':207.020}
    pos_vendorName= {'x0':46.720, 'top':210.052, 'x1':185.400, 'bottom':226.420}
    pos_vendorType={'x0':165.400, 'top':193.080, 'x1':185.400, 'bottom':226.420}
    pos_quo = {'x0':361.800, 'top':190.680, 'x1': 439.800, 'bottom':207.960}
    pos_quoRate = {'x0':420.360, 'top':216.440, 'x1': 439.800, 'bottom':231.320}
    pos_finalPrice = {'x0':635, 'top':190.680, 'x1':725.680 , 'bottom':205.560}
    pos_finalRate = {'x0':710.140, 'top':215.840, 'x1':724.140 , 'bottom':230.720}
    pos_vendorSn = {'x0':20.400, 'top':193.628, 'x1':44.400, 'bottom':215}
    
    #案件資料
    caseID = get_text(p0,pos_caseID)
    caseName = get_text(p0,pos_caseName)
    sn = get_text(p0,pos_sn)
    comp = get_text(p0,pos_comp)
    budget = get_text(p0,pos_budget).replace('$', '')
    caseType = get_text(p0,pos_type)
    currency = get_text(p0,pos_currency)
    #之後要再做一個報價家數的自動計算

    
    #報價資料
    vnd = []
    #之後要做成一個def return，並且append
    vnd.append(getVndData(
        sn = pos_vendorSn,
        id = pos_vendorID,
        name = pos_vendorName,
        type = pos_vendorType,
        quo = pos_quo,
        quoRate = pos_quoRate,
        finalPrice = pos_finalPrice,
        finalRate = pos_finalRate
    ))
    
    #將資料變成一個Bidcase物件
    case= Bidcase(caseID, sn, comp, caseName, budget, caseType, currency, \
              vnd)
    del pdf_1
    del p0
    return case
    
def get_text(pdf, dict_pos):
    crop = pdf.within_bbox((dict_pos['x0'],dict_pos['top'],dict_pos['x1'],dict_pos['bottom']))
    del pdf
    return crop.extract_text()

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier


class Bidcase():
    def __init__(self, ID, sn, comp, name, budget, caseType, currency, \
                 vnd):
        self.id = ID
        self.sn = sn
        self.comp = str(comp)
        self.name = name
        self.budget = budget
        self.currency = currency
        self.vnd = vnd
        
        #vndSn用來計算vnd[]要用第幾筆資料
        self.vndSn = 0
        
       
        if caseType == '一般' and self.sn != '00':
            self.type = '變更案'
        
        elif caseType =='一般' and self.sn == '00' and vnd[self.vndSn]['type'] == '預約':
            self.type = '預約子案'
        
        elif caseType =='補辦':
            self.type = '補辦案'
        
        else:
            self.type = '一般案'
        
    
    #針對沒有議價時較預算差異的計算方式
    def dif_rate(self, vndSn):
        if self.vnd[vndSn]['finalPrice'] == None:
            n1 = self.vnd[vndSn]['quo'].replace(',', '')
        else:
            n1 = self.vnd[vndSn]['finalPrice'].replace(',', '')

        n2 = self.budget.replace(',','')
        dif = round_down((int(n1)/int(n2) -1)*100, 2)
        
        return str(dif) + '%'
    
    def is_equaltoBudget(self, vndSn):
        if self.vnd[vndSn]['finalPrice'] == None:
            return self.vnd[vndSn]['quo'].replace(',', '') == self.budget.replace(',', '')
        else:
            return self.vnd[vndSn]['finalPrice'].replace(',', '') == self.budget.replace(',', '')

    def dif_text(self, vndSn):
        if self.is_equaltoBudget(vndSn):
            return '同預算價格'
        else:
            return '較預算' + self.dif_rate(vndSn)
        
    def submission_text(self, vndSn):
        #initial
        prefixCur =''
        suffixCur = '元'
        prefixVnd = ''
        if self.currency == '人民幣':
            prefixCur = 'RMB'
            suffixCur = ''
        
        def name_format(var):
            return '「' + str(var) +'」'
        
        def price_format(var):
            return prefixCur + str(var) + suffixCur
        
        if self.vnd[vndSn]['finalPrice'] == None:#沒議價時
            award_price = price_format(self.vnd[vndSn]['quo'])
        else:
            award_price = price_format(self.vnd[vndSn]['finalPrice'])
            
        #三段式part1
        if self.type == '變更案':
            prefixVnd = '原承商'
            n = int(self.budget.replace(',', ''))
            if n >= 0:
                goesWay = '追加'
            elif n == 0:
                goesWay = '變更'
            else:
                goesWay = '追減'
        
            part1 = '1.本案為' + self.sn + '次變更案。\n'
            part3 = '3.擬以' + award_price + '向' + name_format(self.vnd[vndSn]['name']) + '辦理' + goesWay + '。\n'
        elif self.type == '預約子案':
            part1 = '1.本案為預約子案，遴選合約商' + name_format(self.vnd[vndSn]['name']) + '施作。\n'
            part3 = '3.擬以' + award_price +'交由' + name_format(self.vnd[vndSn]['name']) + '承攬。\n'
        elif self.type == '補辦案':
            part1 = '1.本案為補辦案，已由' + name_format(self.vnd[vndSn]['name']) + '先行施作。\n'
            part3 = '3.擬以' + award_price +'交由' + name_format(self.vnd[vndSn]['name']) + '承攬。\n'
        else:
            part1 = '1.本案為' + self.comp + name_format(self.name) + '，預算編列' + price_format(self.budget) + '。\n'
        
        #part2
        if self.vnd[vndSn]['finalPrice'] == None:#沒議價時
            part2 = '2.經洽' + prefixVnd + name_format(self.vnd[vndSn]['name']) + '」詢價後報價'\
            +price_format(self.vnd[vndSn]['quo']) + '，' + self.dif_text(vndSn) + '。\n'
        else:#有議價時
            part2 = '2.經洽議' + prefixVnd + '「' + name_format(self.vnd[vndSn]['name']) + '」後由'\
            +price_format(self.vnd[vndSn]['quo']) + '降至' +price_format(self.vnd[vndSn]['finalPrice'])\
            + '，' + self.dif_text(vndSn) + '。\n'
        
        
        return part1+ part2 + part3


if __name__ == '__main__':
    file='d:/s1.pdf'
    myCase = open(file)
    sn = 0
    
    print('工號:', myCase.id)
    print('公司:', myCase.comp)
    print('次別:', myCase.sn)
    print('工程名稱:', myCase.name)
    print('幣別:', myCase.currency)
    print('性質:', myCase.type)
    print('預算:', myCase.budget)
    print('----------------第'+str(sn+1)+'家報價廠商資料-------------')
    print('報價次序:', myCase.vnd[sn]['sn'])
    print('廠商:', myCase.vnd[sn]['name'])
    print('廠商編號:', myCase.vnd[sn]['id'])
    print('報價:', myCase.vnd[sn]['quo'])
    print('議後:', str(myCase.vnd[sn]['finalPrice']))
    print('廠商報價方式:', myCase.vnd[sn]['type'])

    print('\n議價說明:')
    print(myCase.submission_text(sn))


工號: 913031E2
公司: 1PQ
次別: 00
工程名稱: 煉鋼運轉兩處倉庫新增照明
幣別: 人民幣
性質: 預約子案
預算:  29,838
----------------第1家報價廠商資料-------------
報價次序:  1
廠商: 厦门华雅建设
廠商編號: 91350205303102864L
報價:  40,222
議後:  29,838
廠商報價方式: 預約

議價說明:
1.本案為預約子案，遴選合約商「厦门华雅建设」施作。
2.經洽議「「厦门华雅建设」」後由RMB 40,222降至RMB 29,838，同預算價格。
3.擬以RMB 29,838交由「厦门华雅建设」承攬。

